# SLO Plots

In [ ]:
import os
from plot_goodput_vs_slo import read_data, plot_goodput_vs_slo
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'svg')

def run_and_plot(exp_name):
    os.system(f"python gen_data_notebook.py {exp_name}")
    # print(os.popen(f'python run_goodput_experiments.py {exp_name}').read())
    result_file = f"res_{exp_name}.tsv"
    result_plot = f"fig_{exp_name}.pdf"
    data = read_data(result_file)
    plot_goodput_vs_slo(data, result_plot, True)

In [ ]:
run_and_plot("gamma_2")

In [ ]:
run_and_plot("gamma_2_long_slos")

In [ ]:
run_and_plot("gamma_2_short_slos_no_ilp")

In [ ]:
run_and_plot("gamma_2_long_slos_no_ilp")

In [ ]:
experiments = [
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.0",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.05",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.1",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.15",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.2",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.25",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.3",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.35",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.4",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.45",
    "gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.5",
]
for experiment in experiments:
    run_and_plot(experiment)

In [ ]:
datasets = [
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.0.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.05.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.1.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.15.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.2.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.25.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.3.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.35.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.4.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.45.tsv",
    "res_gamma_2_long_slos_no_ilp_duration_400_pipeline_overhead_1.5.tsv",
]

data = {}
for dataset in datasets:
    data[dataset] = read_data(dataset)
sr_data = data[datasets[0]]["sr"]
data = {"mp_" + k.split(".tsv")[0].split("_")[-1]: data[k]["mp"] for k in datasets}
data["sr"] = sr_data

In [ ]:
color_dict = {
    "mp_1.0": "#1f77b4ff",
    "mp_1.05": "#1f77b4ef",
    "mp_1.1": "#1f77b4df",
    "mp_1.15": "#1f77b4cf",
    "mp_1.2": "#1f77b4bf",
    "mp_1.25": "#1f77b4af",
    "mp_1.3": "#1f77b49f",
    "mp_1.35": "#1f77b48f",
    "mp_1.4": "#1f77b47f",
    "mp_1.45": "#1f77b46f",
    "mp_1.5": "#1f77b45f",
    "sr": "C1"
}

def plot_goodput_vs_slo_many(data, output, show):
    fig, ax = plt.subplots()
    figure_size = (4, 4)

    methods = list(data.keys())
    # methods.sort(key=lambda x: method2order(x))

    curves = []
    legends = []
    x_max = 0
    y_max = 0
    for method in methods:
        curve = data[method]
        xs, ys = zip(*curve.items())
        ys = np.array(ys) * 100
        curve = ax.plot(xs, ys, color=color_dict[method], marker='.')
        curves.append(curve[0])
        legends.append(method)

        x_max = max(x_max, *xs)
        y_max = max(y_max, *ys)

    ax.set_ylim(bottom=0, top=max(y_max * 1.05, 100))
    x_max = 64
    ax.set_xlim(left=0, right=x_max * 1.05)
    ax.set_ylabel("Goodput (%)")
    ax.set_xlabel("SLO (second)")
    ax.legend(curves, legends)

    if show:
        plt.show()

    fig.set_size_inches(figure_size)
    fig.savefig(output, bbox_inches='tight')
    print(f"Output the plot to {output}")
plot_goodput_vs_slo_many(data, "fig_pipeline_overheads.pdf", show=True)

# Requests Arrival Pattern

In [ ]:
from alpa_serve.simulator.workload import GammaProcess, UniformMMPP
from alpa_serve.util import GB

In [ ]:
def generate_gamma_arrivals(num_models=16, average_rate=8, cv=10, duration=400, start=0.0, seed=1):
    model_names = [f"m{i}" for i in range(num_models)]
    arrival_process = GammaProcess(average_rate, cv)
    model_arrivals = {}
    for i, name in enumerate(model_names):
        model_arrivals[name] = arrival_process.generate_workload(name, start, duration, seed=seed + i).arrivals
    return model_names, model_arrivals

def generate_uniform_mmpp_arrivals(num_models=16, duration=400, state_durations=(1, 1),
                                   state_request_rates=(2, 4), start=0.0, seed=1):
    model_names = [f"m{i}" for i in range(num_models)]
    arrival_process = UniformMMPP(state_durations, state_request_rates)
    model_arrivals = {}
    for i, name in enumerate(model_names):
        model_arrivals[name] = arrival_process.generate_workload(name, start, duration, seed=seed + i).arrivals
    return model_names, model_arrivals

In [ ]:
model_names, model_arrivals = generate_gamma_arrivals(num_models=16, average_rate=4, cv=10, duration=400)

In [ ]:
for name in model_names[:2]:
    plt.hist(model_arrivals[name], bins=np.linspace(0, 50, 500), alpha=0.8, label=name)
    print(len(model_arrivals[name]) / 400)
plt.title("Sample requests histogram (bin size = 0.1s)")
plt.ylabel("#requests")
plt.xlabel("time (s)")
plt.legend()
fig = plt.gcf()
figure_size = (8, 4)
fig.set_size_inches(figure_size)
fig.savefig("load_samples_50s.pdf", bbox_inches='tight')
fig.show()

In [ ]:
def per_timepoint_average_rate(T, arrivals, timepoints):
    s = 0
    e = 0
    results = []
    for t in timepoints:
        while s < len(arrivals) and arrivals[s] < t - T / 2:
            s += 1
        while e < len(arrivals) and arrivals[e] < t + T / 2:
            e += 1
        results.append((e - s) / T)
    return np.array(results)

In [ ]:
for T in [0.1, 1.0, 4.0, 8.0, 32.0]:
    timepoints = np.linspace(0, 400, 64000)
    average_rates = {}
    for name in model_names:
        average_rates[name] = per_timepoint_average_rate(T, model_arrivals[name], timepoints)
    for name in model_names[:2]:
        plt.plot(timepoints, average_rates[name], label=name)
    plt.plot(timepoints, sum(average_rates.values())/16, label="mean(m0-m15)")
    plt.axhline(4, color='red', label="load=4 requests/s")
    plt.xlim(0, 400)
    plt.xlabel("time (s)")
    plt.ylabel("moving average load (#requests/s)")
    plt.title(f"T={T}s")
    plt.legend()
    plt.axhline(4, color='red')
    fig = plt.gcf()
    figure_size = (8, 4)
    fig.set_size_inches(figure_size)
    fig.savefig(f"average_loads_T{T}.pdf", bbox_inches='tight')
    plt.show()


In [ ]:
def kingmans_formula(arrival_rate, service_rate, cv_arrival, cv_service):
    return arrival_rate / (service_rate - arrival_rate) * (cv_arrival ** 2 + cv_service ** 2) / 2 / service_rate

def deterministic_wait_time(t, D, lambda_):
    rou = lambda_ * D
    res = 0
    for i in range(0, int(np.floor(t/D) + 1)):
        res += np.exp(- lambda_ * (i * D - t)) * (i * D - t) ** i / np.math.factorial(i) * lambda_ ** i
    return res * (1 - rou)

def get_arrival_statistics(arrivals, processing_latency, n_servers=1, 
                           poisson_server=False):
    n_arrivals = len(arrivals)
    arrival_gaps = arrivals[1:] - arrivals[:-1]
    arrival_rate = 1 / np.mean(arrival_gaps)
    cv_arrival = np.std(arrival_gaps) / np.mean(arrival_gaps)

    latencies = []
    current_finishing_time = np.zeros(n_servers)
    wait_times = []
    for arrival in arrivals:
        server_id = np.argmin(current_finishing_time)
        start_time = max(arrival, current_finishing_time[server_id])
        if poisson_server:
            current_finishing_time[server_id] = start_time + np.random.exponential(processing_latency)
        else:
            # Deterministic server
            current_finishing_time[server_id] = start_time + processing_latency
        wait_times.append(start_time - arrival)
        latencies.append(current_finishing_time[server_id] - arrival)
    wait_times = np.array(wait_times)
    latencies = np.array(latencies)

    mean_wait_time = np.mean(wait_times)
    zero_percentage = np.sum(wait_times <= 1e-5) / len(wait_times)
    mean_wait_time_without_zeros = np.mean(wait_times[wait_times > 1e-5])
    
    return (wait_times, latencies, arrival_rate, cv_arrival, 
            mean_wait_time, zero_percentage, mean_wait_time_without_zeros)

def plot_wait_time_pdf_cdf(arrivals, processing_latency, n_servers=1, 
                           poisson_server=False, deterministic_curve_params=None):

    (wait_times, latencies, arrival_rate, cv_arrival, 
     mean_wait_time, zero_percentage, mean_wait_time_without_zeros) = get_arrival_statistics(
        arrivals, processing_latency, n_servers, poisson_server
    )
    
    print("Arrival rate:", arrival_rate)
    print("CV arrival:", cv_arrival)
    print("Mean wait time:", mean_wait_time)
    print("Kingman's prediction:", kingmans_formula(arrival_rate, 1 / processing_latency, cv_arrival, 0.0))
    print("Zero wait time percentage:", zero_percentage)

    # Plot
    x_max = np.max(wait_times)
    x = np.linspace(0, x_max, 100)
    n, bins, _ = plt.hist(wait_times, bins=x, density=True)
    plt.plot(x, 1 / mean_wait_time_without_zeros * np.exp(- x / mean_wait_time_without_zeros))
    plt.title("Wait time PDF")
    plt.xlabel("Wait time (s)")
    plt.show()
    fig = plt.gcf()
    plt.title("Wait time CDF")
    cdfx = np.sort(wait_times)
    cdfy = np.linspace(0, 1, len(wait_times), endpoint=False)
    plt.plot(cdfx, cdfy)
    plt.plot(x, 1 - (1 - zero_percentage) * np.exp(- x / np.mean(mean_wait_time_without_zeros)))
    if deterministic_curve_params is not None:
        D, lambda_ = deterministic_curve_params
        cdfy_estimated = [deterministic_wait_time(x, D, lambda_) for x in cdfx]
        plt.plot(cdfx, cdfy_estimated)
    plt.axhline(0.99, color='red', linewidth=0.5)
    plt.axhline(0.95, color='red', linewidth=0.5)
    plt.xlabel("Wait time (s)")
    plt.show()

    return cdfx, cdfy

In [ ]:
model_names, model_arrivals = generate_uniform_mmpp_arrivals(
    num_models=1, duration=50000, state_durations=(100, 1), state_request_rates=(4, 400), seed=2)
arrivals = np.array(model_arrivals["m0"])
processing_latency = 0.1

In [ ]:
cdfx1, cdfy1 = plot_wait_time_pdf_cdf(arrivals, processing_latency)

In [ ]:
cdfx10, cdfy10 = plot_wait_time_pdf_cdf(arrivals, processing_latency * 10, n_servers=10)
plt.plot(cdfx1, cdfy1, label="1 server latency = 0.1")
plt.plot(cdfx10, cdfy10, label="10 server latency = 1.0")
plt.legend()
plt.show()

In [ ]:
_ = plt.hist(arrivals, bins=np.linspace(0, 1000, 1000))

In [ ]:
model_names, model_arrivals = generate_uniform_mmpp_arrivals(
    num_models=1, duration=50000, state_durations=(10, 1), state_request_rates=(4, 40), seed=2)
arrivals = np.array(model_arrivals["m0"])
processing_latency = 0.1
_ = plot_wait_time_pdf_cdf(arrivals, processing_latency)

In [ ]:
_ = plt.hist(arrivals, bins=np.linspace(0, 1000, 1000))

In [ ]:
model_names, model_arrivals = generate_gamma_arrivals(
    num_models=1, average_rate=8, cv=1, duration=100000, seed=2)
arrivals = np.array(model_arrivals["m0"])
processing_latency = 0.1
_ = plot_wait_time_pdf_cdf(arrivals, processing_latency)

In [ ]:
model_names, model_arrivals = generate_gamma_arrivals(
    num_models=1, average_rate=8, cv=1, duration=100000, seed=2)
arrivals = np.array(model_arrivals["m0"])
processing_latency = 0.1
_ = plot_wait_time_pdf_cdf(arrivals, processing_latency, poisson_server=True)

In [ ]:
_ = plt.hist(arrivals, bins=np.linspace(0, 1000, 1000))

In [ ]:
average_rate = 5
model_names, model_arrivals = generate_gamma_arrivals(
    num_models=1, average_rate=average_rate, cv=1, duration=100000, seed=2)
arrivals = np.array(model_arrivals["m0"])
processing_latency = 0.1
# _ = plot_wait_time_pdf_cdf(arrivals, processing_latency, n_servers=1, 
#                            deterministic_curve_params=(processing_latency, average_rate))
_ = plot_wait_time_pdf_cdf(arrivals, processing_latency, n_servers=1)

In [ ]:
_ = plt.hist(arrivals, bins=np.linspace(0, 1000, 1000))

In [ ]:
def test_with_multiple_processing_latencies(arrivals):
    arrival_gaps = arrivals[1:] - arrivals[:-1]
    arrival_rate = 1 / np.mean(arrival_gaps)

    print(f"arrival_rate={arrival_rate}")

    processing_latencies = np.linspace(0.01 * 1 / arrival_rate, 0.99 * 1 / arrival_rate, 40)
    zero_percentage_list = []
    mean_wait_time_without_zeros_list = []
    for processing_latency in processing_latencies:
        (wait_times, latencies, arrival_rate_, cv_arrival, 
         mean_wait_time, zero_percentage, mean_wait_time_without_zeros) = get_arrival_statistics(
            arrivals, processing_latency, 
            # poisson_server=True
        )
        zero_percentage_list.append(zero_percentage)
        mean_wait_time_without_zeros_list.append(mean_wait_time_without_zeros)
        print(f"processing_latency={processing_latency}, "
              f"zero_percentage={zero_percentage}, "
              f"mean_wait_time_without_zeros={mean_wait_time_without_zeros}")
    return arrival_rate, processing_latencies, zero_percentage_list, mean_wait_time_without_zeros_list

In [ ]:
mmpp_arrivals = np.array(generate_uniform_mmpp_arrivals(
    num_models=1, duration=50000, state_durations=(10, 1), state_request_rates=(4, 40), seed=3)[1]["m0"])
mmpp_simulated_results = test_with_multiple_processing_latencies(mmpp_arrivals)

In [ ]:
poisson_arrivals = np.array(generate_gamma_arrivals(
    num_models=1, average_rate=5, cv=1, duration=50000, seed=2)[1]["m0"])
poisson_simulated_results = test_with_multiple_processing_latencies(poisson_arrivals)

In [ ]:
gamma_arrivals = np.array(generate_gamma_arrivals(
    num_models=1, average_rate=8, cv=3, duration=50000, seed=2)[1]["m0"])
gamma_simulated_results = test_with_multiple_processing_latencies(gamma_arrivals)

In [ ]:
def plot_linear_fit(x, y, fit=True, fit_with_first_datapoints=False):
    sorted_indices = np.argsort(x)
    x = x[sorted_indices][:-10]
    y = np.array(y)
    y = y[sorted_indices][:-10]
    plt.plot(x, y, '.-')
    if fit:
        if fit_with_first_datapoints:
            k, b = np.polyfit(x[:20], y[:20], deg=1)
        else:
            k, b = np.polyfit(x, y, deg=1)
        plt.plot(x, k * x + b, '.-')
        print(f"k={k}, b={b}")
    plt.show()

def visualize_results(result, fit=True):
    arrival_rate, processing_latencies, zero_percentage_list, mean_wait_time_without_zeros_list = result    
    processing_latencies = np.array(processing_latencies)
    zero_percentage_list = np.array(zero_percentage_list)
    mean_wait_time_without_zeros_list = np.array(mean_wait_time_without_zeros_list)
    alpha = 1 - zero_percentage_list
    beta = 1 / mean_wait_time_without_zeros_list
    # Plot 1
    print("alpha")
    x = np.concatenate(([0.0], (arrival_rate * processing_latencies))) 
    y = np.concatenate(([0.0], alpha))
    plot_linear_fit(x, y, fit)
    # Plot 2
    print("beta")
    x = (1 / processing_latencies - arrival_rate)
    y = beta
    plot_linear_fit(x, y, fit)
    # Plot 3
    print("alpha vs. beta")
    plot_linear_fit((1 - alpha) / processing_latencies, beta, fit)
    plt.show()

In [ ]:
print("-" * 40 + " Poisson results " + "-" * 40)
visualize_results(poisson_simulated_results, fit=True)
print("-" * 40 + "  MMPP results   " + "-" * 40)
visualize_results(mmpp_simulated_results, fit=True)
print("-" * 40 + "  Gamma results  " + "-" * 40)
visualize_results(gamma_simulated_results, fit=True)

In [ ]:
def gamma_r0(k, lambda_, mu):
    p = np.zeros(k + 2)
    p[0] = mu
    p[-2] = -(k * lambda_ + mu)
    p[-1] = k * lambda_
    roots = np.roots(p)
    for root in roots:
        if np.isreal(root) and 0 < root < 1-1e-10:
            return root
    print(k, lambda_, mu)
    print(k, lambda_, mu)
    print(roots)
    raise ValueError()


lambda_ = 4.0
mus = np.linspace(lambda_ + 1, lambda_ * 10)
for k in [2, 3, 4]:
    r0s = []    
    for mu in mus:
        r0s.append(gamma_r0(k, lambda_, mu))
    r0s = np.array(r0s)
    plt.plot(mus, r0s ** k)

# Sketch

In [ ]:
import pickle
from alpa_serve.profiling import ParallelConfig
from alpa_serve.placement_policy import ModelParallelismPlacement
import numpy as np

In [ ]:
result = pickle.load(open("profiling_result.pkl", "rb"))

In [ ]:
latency_8stages = np.array(result['bert-1.3b'].para_dict[ParallelConfig(1, 1, 8)].latency[1])

In [ ]:
latency_8stages.max() * 8

In [ ]:
latency_1stage = np.array(result['bert-1.3b'].para_dict[ParallelConfig(1, 1, 1)].latency[1])

In [ ]:
latency_1stage

In [ ]:
0.12776608 / 0.0996182

In [ ]:
np.array(result['bert-1.3b'].para_dict[ParallelConfig(1, 1, 4)].latency[1]).max() * 4

In [ ]:
0.10797936 / 0.0996182

In [ ]:
def compute_capability(slo, profile_result, parallel_config):
    slo = slo
    latency_mem = profile_result.para_dict.get(parallel_config, None)

    if latency_mem is None:
        return 0

    num_stages = parallel_config.pp
    max_cap = 0
    max_bs = 1
    for b, ls in latency_mem.latency.items():
        if b > max_bs:
            continue

        # slo = sum(ls) + (n-1) * max(ls)
        # so, n = ceil((slo - sum(ls)) / max(ls)) + 1
        max_cap = max(max_cap, (slo - sum(ls)) // max(ls) + 1)

    return max_cap


In [ ]:
slos = [0.1, 0.2, 0.4, 0.8, 1.0, 2.0, 4.0, 8.0]
for slo in slos:
    for pp in [1, 2, 4, 8]:
        print("slo", slo, "pp", pp, compute_capability(slo, result['bert-1.3b'], ParallelConfig(1, 1, pp)))

In [ ]:
# slo 0.8 pp 1 8.0
# slo 0.8 pp 2 15.0
# slo 0.8 pp 4 26.0
# slo 0.8 pp 8 44.0
# slo 1.0 pp 1 10.0
# slo 1.0 pp 2 19.0
# slo 1.0 pp 4 34.0
# slo 1.0 pp 8 56.0

# SLO 0.8: Optimal case
# ModelParallelismPlacement([(ParallelConfig(dp=1, op=1, pp=4), [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15]), 
#                            (ParallelConfig(dp=1, op=1, pp=4), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15])])
obj_optimal = 26 * 2
obj_non_optimal = 26
# SLO 1.0: Optimal case
# ModelParallelismPlacement([(ParallelConfig(dp=1, op=1, pp=2), [0, 7, 9, 10, 11, 12, 13]), 
#                            (ParallelConfig(dp=1, op=1, pp=1), [7, 11, 12]), 
#                            (ParallelConfig(dp=1, op=1, pp=1), [2, 5, 7]), 
#                            (ParallelConfig(dp=1, op=1, pp=4), [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15])])
obj_optimal = 34.0 * 2
obj_non_optimal = 34.0

In [ ]:
result['bert-1.3b'].para_dict[ParallelConfig(1, 1, 1)].weight_mem

In [ ]:
from alpa_serve.util import GB

In [ ]:
10*GB

In [ ]:
40 / (2838835202.8206697 / GB)

In [ ]:
len([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15])